In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-3.0.0-preview-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('International_Trip').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
spark

In [4]:
#Import Library
#import findspark
#findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
from pyspark.sql.functions import month
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.functions import round,array,col,ntile,rank,asc,sum,desc,lag,lead,when,first,min,max,count
# For Adding row number/index
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
import pyspark
from pyspark import SparkContext
from pyspark import SparkContext, SparkConf, keyword_only
from pyspark.sql import SparkSession
#from pyspark.sql import SQLContext, HiveContext, SparkSession
from pyspark.sql import Row

In [5]:
# spark = SparkSession.builder.appName('international_trip')\
# .config("spark.driver.memory", "4g")\
# .config('spark.executor.cores','4')\
# .config('spark.executor.memory','12g')\
# .config('spark.sql.broadcastTimeout','2000')\
# .config("spark.sql.autoBroadcastJoinThreshold", -1)\
# .getOrCreate()

# spark

In [5]:
# import data
cc_ndc=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\cc_ndc.csv",inferSchema='true')
cell_mr=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\Cell_mr_data_nov.csv",inferSchema='true')
Roming_data=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\Roming_data_nov.csv",inferSchema='true')
Simtofly=spark.read.option('header','true').csv(r"C:\Users\harin\Desktop\international\Raw_Data\Simtofly_nov.csv",inferSchema='true')

In [11]:
cell_mr1=cell_mr.withColumn('time_in',when(col('time_in').contains("00:00"),col('time_out')).otherwise(col('time_in')))

In [15]:
import pyspark.sql.functions as F
cell_mr1=cell_mr1.withColumn('time_in',F.to_timestamp(col('time_in'), 'dd-MM-yyyy HH:mm'))\
            .withColumn('time_out',F.to_timestamp(col('time_out'), 'dd-MM-yyyy HH:mm'))

In [19]:
#cell_mr2=cell_mr1.filter(col("time_in").contains("00:00:00"))
# cell_mr3=cell_mr1.where(~col('time_in').contains("00:00"))

In [11]:
# cell_mr2=cell_mr2.withColumnRenamed("time_in_dt","time_out_dt1").withColumnRenamed("time_out_dt","time_in_dt")
# cell_mr2=cell_mr2.withColumnRenamed("time_out_dt1","time_out_dt")

# cell_mr4 = cell_mr2.union(cell_mr3)
# #cell_mr4.count()

In [21]:
Roming_data=Roming_data.withColumn('partition_dt',to_timestamp(Roming_data.partition_date, 'dd-MM-yyyy'))\
            .withColumn('activetime_dt',to_timestamp(Roming_data.activetime, 'dd-MM-yyyy HH:mm'))

Roming_data=Roming_data.withColumn('activetime_month',month(Roming_data.activetime_dt))\
.withColumn('partition_month',month(Roming_data.partition_dt))
#Roming_data.count()

In [22]:
Roming_data1=Roming_data.filter(Roming_data.activetime_month==Roming_data.partition_month)
Roming_data2=Roming_data1.join(cc_ndc,(Roming_data1.visitcc==cc_ndc.cc) & (Roming_data1.visitndc==cc_ndc.ndc),'left' )

In [23]:
Roming_data3=Roming_data2.select('batchtime','activetime','imsi','c360_subscription_identifier',
                                 'msisdn','mcc','mnc','visitcc','visitndc','partition_date','partition_dt',
                                 'activetime_dt','activetime_month','partition_month','cc','ndc','operator_name',
                                 'country_name')
Roming_data3=Roming_data3.dropDuplicates()
#Roming_data3.count()

In [24]:
Roming_Simtofly=Roming_data3.join(Simtofly.select('imsi','imsi_sim2fly'),['imsi'],'left')
#Roming_Simtofly.count()

In [25]:
Roming_Simtofly = Roming_Simtofly.withColumn("imsi", when(col("imsi_sim2fly").isNull(),col("imsi")
                                                         ).otherwise(col("imsi_sim2fly")))
#Roming_Simtofly.count()

In [26]:
cell_mr1=cell_mr1.withColumn("country_name", lit('thailand'))
#cell_mr4.count()

In [27]:
Roming_Simtofly=Roming_Simtofly.withColumnRenamed("activetime","time_in")
#Roming_Simtofly.count()

In [39]:
Roming_Simtofly=Roming_Simtofly.withColumn('time_in',to_timestamp(col('time_in'), 'dd-MM-yyyy HH:mm'))

In [40]:
from pyspark.sql import DataFrame
from pyspark.sql import functions as F

def add_missing_columns(df: DataFrame, ref_df: DataFrame) -> DataFrame:
    for col in ref_df.schema:
        if col.name not in df.columns:
            df = df.withColumn(col.name, F.lit(None).cast(col.dataType))

    return df

df1 = add_missing_columns(cell_mr1, Roming_Simtofly)
df2 = add_missing_columns(Roming_Simtofly, cell_mr1)

df_result = df1.unionByName(df2)
df_result.count()

7500

In [29]:
# df_result=df_result.withColumn('time_in',F.to_timestamp(df_result.time_in, 'dd-MM-yyyy HH:mm'))\
# .withColumn('time_out',F.to_timestamp(df_result.time_out, 'dd-MM-yyyy HH:mm'))
# #df_result.count()

In [42]:
df=df_result.select(['imsi','grid_id','grid_name','grid_latitude','grid_longitude','source_flag','region_id','region_name',
 'sub_region_id','sub_region_name','province_id','province_name','district_id','district_name','sub_district_id','sub_district_name','time_in',
 'time_out','hour_in','hour_out','duration','country_network_code','c360_subscription_identifier','inbound_flag','indoor_flag',
 'cell_ids','country_name','operator_name','mcc','mnc','cc','ndc','imsi_sim2fly']) 
df=df.dropDuplicates()
df.count()

6410

In [44]:
df.repartition(1).write.option("header",True).mode("overwrite").csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\international_trip_data")

In [45]:
unique_imsi = df.select('imsi').distinct()
unique_imsi = unique_imsi.withColumn('time_in', F.lit("2000-01-01 00:00:00"))
unique_imsi = unique_imsi.withColumn('country_name', F.lit("thailand"))
#unique_imsi.show()

In [46]:
missing_columns = set(df.columns) - set(unique_imsi.columns)
for column in missing_columns:
    unique_imsi = unique_imsi.withColumn(column, F.lit(None))

In [47]:
df = df.unionByName(unique_imsi)
#df.orderBy("time_in").show(5,vertical=True)

In [48]:
unique_imsi = unique_imsi.withColumn('time_in', F.lit('null'))
df = df.unionByName(unique_imsi)
df = df.orderBy("time_in")
#df.show(5,vertical=True)

In [49]:
from pyspark.sql.functions import lag  
from pyspark.sql.functions import lead 
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql import functions as f

windowSpec  = Window.orderBy("time_in")

df = df.withColumn("index",row_number().over(windowSpec))


In [50]:
df=df.withColumn('time_in',F.to_timestamp(df.time_in, 'yyyy-MM-dd HH:mm:ss'))
#df.toPandas().to_csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\raw_data.csv")

In [51]:
df=df.withColumn('country_name', f.when((f.col('country_name').isNull()),'Data_not_Available').otherwise(f.col('country_name')))


In [52]:
#df.filter(col('imsi')=='56aTd4iFWJXhSdDlfpcK0EY2CfkKyBTsKZpo.cjMfs++SDp6yfap97HBQJtSIphC').toPandas().to_csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\test_data.csv")

In [53]:
windowSpec  = Window.partitionBy("imsi").orderBy("time_in")

#shift -1 country
df = df.withColumn("lag_country",lag("country_name",1).over(windowSpec)) 

#shift -1 country with only thailand
#df = df.withColumn('lag_country_thailand', f.when(f.col('lag_country') == 'thailand', lit("thailand")).otherwise(lit(None)))
df = df.withColumn("trip_start_id", F.lit(None))

df = df.withColumn('trip_start', f.when((f.col('country_name')!='thailand')&(f.col('lag_country')=='thailand') , lit("start")).otherwise(lit(None)))

trip_start = df.filter(df.trip_start=='start')
print(trip_start.count())

windowSpec  = Window.partitionBy("imsi").orderBy("time_in")
trip_start = trip_start.withColumn("trip_start_id",row_number().over(windowSpec))

df = df.filter(df.trip_start.isNull())
#print(df.count())

df = df.unionByName(trip_start)
#print(df.count())


9


In [54]:
windowSpec  = Window.partitionBy("imsi").orderBy("time_in")

#shift +1 country
df = df.withColumn("lead_country",lead("country_name",1).over(windowSpec))

#shift +1 country with only thailand
#df = df.withColumn('lead_country_thailand', f.when(f.col('lead_country') == 'thailand', lit("thailand")).otherwise(lit(None)))
df = df.withColumn("trip_end_id", F.lit(None))

df = df.withColumn('trip_end', f.when((f.col('country_name')!='thailand')&(f.col('lead_country')=='thailand'), lit("end")).otherwise(lit(None)))

trip_end = df.filter(df.trip_end=='end')
#print(trip_end.count())

windowSpec  = Window.partitionBy("imsi").orderBy("time_in")
trip_end = trip_end.withColumn("trip_end_id",row_number().over(windowSpec))

df = df.filter(df.trip_end.isNull())
#print(df.count())

df = df.unionByName(trip_end)
#print(df.count())


In [55]:
df = df.withColumn("trip_flag",f.when(f.col('trip_start_id').isNotNull(), lit(1)).otherwise(lit(0)))
#print(df.count())

In [56]:
windowSpec  = Window.partitionBy("imsi").orderBy("time_in")
df = df.withColumn('trip_start_cumsum', f.sum(f.col('trip_flag')).over(windowSpec))
#print(df.count())


In [57]:
#if country != lag_country and country !=thailand
df = df.withColumn('sub_start', f.when((f.col('country_name') != f.col('lag_country'))&(f.col('country_name') != 'thailand') , lit("start")).otherwise(lit(None)))
df = df.withColumn("start_sub_trip_id", F.lit(None))

sub_trip_start = df.filter(df.sub_start=='start')
#print(sub_trip_start.count())


windowSpec  = Window.partitionBy("imsi","trip_start_cumsum").orderBy("time_in")
sub_trip_start = sub_trip_start.withColumn("start_sub_trip_id",row_number().over(windowSpec))

df = df.filter(df.sub_start.isNull())
#print(df.count())

df = df.unionByName(sub_trip_start)
#print(df.count())


In [58]:
df = df.withColumn("sub_trip_start_flag",f.when(f.col('start_sub_trip_id').isNotNull(), lit(1)).otherwise(lit(0)))

In [59]:
windowSpec  = Window.partitionBy("imsi").orderBy("time_in")
df = df.withColumn('sub_trip_start_cumsum', f.sum(f.col('sub_trip_start_flag')).over(windowSpec))

In [60]:
operator_name = df.groupBy("imsi","trip_start_cumsum","sub_trip_start_cumsum").agg(collect_list(col("operator_name")).alias('operator'))

In [61]:
from pyspark.sql.functions import array_distinct
operator_name1 = operator_name.withColumn("operator", array_distinct("operator"))

In [62]:
#if country != lead_country and country !=thailand
df = df.withColumn('sub_end', f.when((f.col('country_name') != f.col('lead_country'))&(f.col('country_name') != 'thailand') , lit("end")).otherwise(lit(None)))
df = df.withColumn("end_sub_trip_id", F.lit(None))

sub_trip_end = df.filter(df.sub_end=='end')
#print(sub_trip_end.count())


windowSpec  = Window.partitionBy("imsi","trip_start_cumsum").orderBy("time_in")
sub_trip_end = sub_trip_end.withColumn("end_sub_trip_id",row_number().over(windowSpec))

df = df.filter(df.sub_end.isNull())
#print(df.count())

df = df.unionByName(sub_trip_end)
#print(df.count())


In [63]:
windowSpec  = Window.partitionBy("imsi").orderBy("time_in")

#thailand's last sample time_out
df = df.withColumn("trip_start_time",lag("time_out",1).over(windowSpec))

#thailand's first sample(after trip) time_in 
df = df.withColumn("trip_end_time",lead("time_in",1).over(windowSpec)) 


In [64]:
from pyspark.sql.functions import col
# for trip
trip_start = df.filter(df.trip_start_id.isNotNull()).select('imsi',col('trip_start_id').alias('trip_id'),'trip_start_time')
trip_end = df.filter(df.trip_end_id.isNotNull()).select('imsi',col('trip_end_id').alias('trip_id'),'trip_end_time')

# for sub_trip
sub_trip_start = df.filter(df.start_sub_trip_id.isNotNull()).select('imsi',col('trip_start_cumsum').alias('trip_id'),col('start_sub_trip_id').alias('subtrip_id'),'time_in',col('country_name'))
sub_trip_end = df.filter(df.end_sub_trip_id.isNotNull()).select('imsi',col('trip_start_cumsum').alias('trip_id'),col('end_sub_trip_id').alias('subtrip_id'),col('time_in').alias('time_out'))


In [65]:
#df.toPandas().to_csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_start.csv")

In [66]:
#trip_end.toPandas().to_csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\trip_end.csv")

In [67]:
#sub_trip_start.toPandas().to_csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\sub_trip_start.csv")

In [68]:
#sub_trip_end.toPandas().to_csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\sub_trip_end.csv")

## Trip level

In [69]:
trip_start = trip_start.withColumn("trip_start_time", f.when(col("trip_start_time").isNull(), "Trip_start_missing").otherwise(col("trip_start_time")))
trip_end = trip_end.withColumn("trip_end_time", f.when(col("trip_end_time").isNull(), "Trip_end_missing").otherwise(col("trip_end_time")))
sub_trip_start = sub_trip_start.withColumn("time_in", f.when(col("time_in").isNull(), "Trip_start_missing").otherwise(col("time_in")))
sub_trip_end = sub_trip_end.withColumn("time_out", f.when(col("time_out").isNull(), "Trip_end_missing").otherwise(col("time_out")))

In [70]:
trip_level = trip_start.join(trip_end, ['imsi','trip_id'], how='outer').orderBy('imsi','trip_id')
country_level = sub_trip_start.join(sub_trip_end, ['imsi','trip_id','subtrip_id'], how='outer').orderBy('imsi','trip_id','subtrip_id')

In [71]:
df_result=df_result.select(['imsi','grid_id','grid_name','grid_latitude','grid_longitude','source_flag','region_id','region_name',
 'sub_region_id','sub_region_name','province_id','province_name','district_id','district_name','sub_district_id','sub_district_name','time_in',
 'time_out','hour_in','hour_out','duration','country_network_code','c360_subscription_identifier','inbound_flag','indoor_flag',
 'cell_ids','country_name','operator_name','mcc','mnc','cc','ndc','imsi_sim2fly']) 
df_result=df_result.dropDuplicates()

In [72]:
#joining with grid Id & Grid name 
trip_level1 = trip_level.select("imsi", "trip_id", "trip_start_time","trip_end_time").alias("a")\
.join(df_result.select('imsi','grid_id','grid_name','grid_latitude','grid_longitude','time_in','time_out').dropDuplicates(subset = ["imsi","time_out"]).alias("b"),
                             (trip_level.imsi==df_result.imsi) & (trip_level.trip_start_time==df_result.time_out),'left')\
.select("a.imsi", "trip_id", "trip_start_time","trip_end_time",'grid_id','grid_name','grid_latitude','grid_longitude')

In [73]:
trip_level1=trip_level1.withColumnRenamed("grid_id","exit_grid_id").withColumnRenamed("grid_name","exit_grid_name")\
.withColumnRenamed("grid_latitude","exit_grid_latitude").withColumnRenamed("grid_longitude","exit_grid_longitude")

In [74]:
trip_level2= trip_level1.alias("a")\
.join(df_result.select('imsi','grid_id','grid_name','grid_latitude','grid_longitude','time_in','time_out').dropDuplicates(subset = ["imsi","time_in"]).alias("b"),
                             (trip_level.imsi==df_result.imsi) & (trip_level.trip_end_time==df_result.time_in),'left')\
.select('a.imsi','trip_id','trip_start_time','trip_end_time','exit_grid_id','exit_grid_name',
        'exit_grid_latitude','exit_grid_longitude','grid_id','grid_name','grid_latitude','grid_longitude',)

In [75]:
trip_level2=trip_level2.withColumnRenamed("grid_id","entry_grid_id").withColumnRenamed("grid_name","entry_grid_name")\
.withColumnRenamed("grid_latitude","entry_grid_latitude").withColumnRenamed("grid_longitude","entry_grid_longitude")

In [76]:
trip_level2=trip_level2.withColumn("trip_end_time", when(col("trip_end_time").isNull(),'Trip_end_missing'
                                                         ).otherwise(col("trip_end_time")))

In [77]:
#trip_level3=trip_level2.filter(trip_level2.trip_start_time!=trip_level2.trip_end_time)

## Sub Trip level

In [78]:
# #adding cuntory and oprater 
# country_level=country_level.select("imsi", "trip_id",'subtrip_id', 'time_in', 'time_out' ).alias("a")\
# .join(df_result.select('imsi','country_name','operator_name','time_in').dropDuplicates(subset = ["imsi","time_in"]).alias("b"),
#                              ["imsi","time_in"],'left')\
# .select("imsi", "trip_id",'subtrip_id', 'time_in', 'time_out','country_name','operator_name')


In [79]:
country_level1=country_level.alias("a").join(operator_name1.alias("b"),(country_level.imsi==operator_name1.imsi) & (country_level.subtrip_id==operator_name1.sub_trip_start_cumsum),'left').select('a.imsi','trip_id','subtrip_id','time_in','time_out','country_name','operator')

In [80]:
country_level1=country_level1.withColumn('operator',col('operator').cast('String'))

In [81]:
country_level1=country_level1.withColumn("time_out", when(col("time_out").isNull(),'Trip_end_missing'
                                                         ).otherwise(col("time_out")))

In [82]:
#country_level1=country_level1.filter(country_level1.time_in!=country_level1.time_out)

In [83]:
country_travel=country_level1.groupby('imsi','trip_id').agg(count(col("country_name")).alias('No_country_name_visit'))

In [84]:
trip_level3=trip_level2.join(country_travel,['imsi','trip_id'],'left')

In [85]:
#df.repartition(1).write.option("header",True).mode("overwrite").csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\international_trip_data")

In [86]:
trip_level3.repartition(1).write.option("header",True).mode("overwrite").csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\international_trip_nov")

In [ ]:
country_level1.repartition(1).write.option("header",True).mode("overwrite").csv(r"C:\Users\harin\Desktop\international\Raw_Data\test_output\international_sub_trip_nov")

In [ ]:
spark.stop()